# Spotify Popularity Predictor (39%)

The goal of this challenge is to create a model that predicts the popularity of a song based on its features.

The dataset contains a list of tracks with the following characteristics:
- `acousticness`: whether the track is acoustic
- `danceability`: describes how suitable a track is for dancing
- `duration_ms`: duration of the track in milliseconds
- `energy`: represents a perceptual measure of intensity and activity
- `explicit`: whether the track has explicit lyrics
- `id`: id for the track
- `instrumentalness`: predicts whether a track contains no vocals
- `key`: the key the track is in
- `liveness`: detects the presence of an audience in the recording
- `loudness`: the overall loudness of a track in decibels
- `mode`: modality of a track
- `name`: name of the track
- `popularity`: popularity of the track
- `release_date`: release date
- `speechiness`: detects the presence of spoken words in a track
- `tempo`: overall estimated tempo of a track in beats per minute
- `valence`: describes the musical positiveness conveyed by a track
- `artist`: artist who performed the track

# Model

## Data collection

**📝 Load the `spotify_popularity_train.csv` dataset from the provided URL. Display the first few rows. Perform the usual cleaning operations. Store the result in a `DataFrame` named `data`.**

👉 Do not forget to clean the dataset

In [66]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import make_column_transformer

In [2]:
url = "https://wagon-public-datasets.s3.amazonaws.com/certification_paris_2021Q1/spotify_popularity_train.csv"

In [3]:
data = pd.read_csv(url)
data.head(3)

,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,artist
0,0.65400,0.499,219827,0.190,0,0B6BeEUd6UwFlbsHMQKjob,0.00409,7,0.0898,-16.435,1,Back in the Goodle Days,40,1971,0.0454,149.460,0.4300,John Hartford
1,0.00592,0.439,483948,0.808,0,5Gpx4lJy3vKmIvjwbiR5c8,0.14000,2,0.0890,-8.497,1,Worlds Which Break Us - Intro Mix,22,2015-02-02,0.0677,138.040,0.0587,Driftmoon
2,0.73400,0.523,245693,0.288,0,7MxuUYqrCIy93h1EEHrIrL,0.00000,0,0.0771,-11.506,1,I'm The Greatest Star,40,1968-09-01,0.2140,75.869,0.4640,Barbra Streisand


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52317 entries, 0 to 52316
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      52317 non-null  float64
 1   danceability      52317 non-null  float64
 2   duration_ms       52317 non-null  int64  
 3   energy            52317 non-null  float64
 4   explicit          52317 non-null  int64  
 5   id                52317 non-null  object 
 6   instrumentalness  52317 non-null  float64
 7   key               52317 non-null  int64  
 8   liveness          52317 non-null  float64
 9   loudness          52317 non-null  float64
 10  mode              52317 non-null  int64  
 11  name              52317 non-null  object 
 12  popularity        52317 non-null  int64  
 13  release_date      52317 non-null  object 
 14  speechiness       52317 non-null  float64
 15  tempo             52317 non-null  float64
 16  valence           52317 non-null  float6

In [5]:
# drop duplicates, drop null values
data = data.drop_duplicates().dropna()

In [6]:
# scale values
scaler = MinMaxScaler()
features_to_scale = ['acousticness', 
                   'danceability', 
                   'duration_ms',
                   'energy', 
                   'explicit', 
                   'instrumentalness', 
                   'liveness', 
                   'loudness',
                   'mode',
                   'speechiness',
                   'tempo',
                   'valence'
                  ]
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

### Save your results

Run the following cell to save your results.

In [7]:
from nbresult import ChallengeResult

ChallengeResult(
    "c5_data_cleaning",
    data=data).write()

## Baseline

**📝 We want to use a metric that measures the prediction error in the same unit than `popularity`. In addition, it should strongly penalize largest errors. Which sklearn's [metric](https://scikit-learn.org/stable/modules/model_evaluation.html) should we use? Store its exact name as string below**

In [8]:
scoring = "neg_mean_squared_error"

**📝 Let's build a baseline model using only the numerical features in our dataset.**
- Build `X_baseline` with only numerical features
- Build `y` your target containing the `popularity`
- Then 5 times cross validate the baseline linear model of your choice (do not fine tune it)
- Store your mean performance in a `float` variable named `baseline_score`

In [9]:
X_baseline = data[['acousticness', 
                   'danceability', 
                   'duration_ms', 
                   'energy', 
                   'explicit', 
                   'instrumentalness', 
                   'liveness', 
                   'loudness',
                   'mode',
                   'speechiness',
                   'tempo',
                   'valence'
                  ]]

y = data['popularity']

In [10]:
baseline_model = DummyRegressor(strategy="median")
cv_results = cross_validate(baseline_model, X_baseline, y, cv=5, scoring=scoring)
baseline_score = cv_results['test_score'].mean()

### Save your results

Run the following cell to save your results.

In [11]:
from nbresult import ChallengeResult

ChallengeResult(
    "baseline",
    scoring=scoring,
    baseline_score=baseline_score).write()

## Feature engineering

Let's now use the features that we left aside: `release_date` and `artist` to improve the performance of our model. We'll create them manually in a train vs. test context first (and pipeline them later)

### holdout
**📝 Create the 4 variables `X_train` `y_train`, `X_test`, `y_test` with a 50% split with random sampling**

In [12]:
X = data[['acousticness', 
           'danceability', 
           'duration_ms', 
           'energy', 
           'explicit', 
           'instrumentalness', 
           'liveness', 
           'loudness',
           'mode',
           'speechiness',
           'tempo',
           'valence',
           'release_date',
           'artist',
          ]]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

### year

**📝 Create `X_train_year` and `X_test_year` by adding the new column `year` containing the release year of the track as integer**

In [18]:
X_train_year = X_train.copy()
X_train_year['year'] = pd.to_datetime(X_train_year['release_date']).dt.year

In [19]:
X_test_year = X_test.copy()
X_test_year['year'] = pd.to_datetime(X_test_year['release_date']).dt.year

In [20]:
X_train_year.head()

,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,mode,speechiness,tempo,valence,release_date,artist,year
27694,0.996988,0.639959,0.025517,0.0346,0.0,0.959,0.086486,0.608198,1.0,0.051753,0.346581,0.3600,2016-10-28,Donald Shoenberg,2016
50854,0.692771,0.717039,0.060294,0.3460,0.0,0.145,0.100100,0.746784,0.0,0.025876,0.349452,0.4900,1985,Eagles,1985
11557,0.888554,0.184584,0.013900,0.0331,0.0,0.818,0.656657,0.538702,1.0,0.039278,0.446969,0.0376,1948,Igor Stravinsky,1948
42812,0.950803,0.235294,0.028197,0.0303,0.0,0.860,0.090691,0.725260,1.0,0.037526,0.422592,0.0366,2014,William Elliott Whitmore,2014
26106,0.668675,0.637931,0.062163,0.5050,0.0,0.000,0.359359,0.840252,1.0,0.057629,0.549159,0.1780,2013-11-07,เบนซ์-พริกไทย,2013


### artist

How could we use the `artist` column? There are too many artists to one hot encode it.  
We could instead create an `artist_popularity` feature containing the mean popularity of an artist, computed as the mean popularity of all tracks the artist released _on the train set_.

#### Process artist popularity from the Training set

**📝 Compute and store the `artist_popularity` as a new pandas `Series`**  

In [26]:
train_set = pd.concat([X_train, y_train], axis=1)

In [27]:
artist_popularity = train_set[['artist', 'popularity']].groupby(['artist']).mean()

#### Apply the artist popularity to `X_train_year`

**📝 Create a new DataFrame `X_train_engineered` which adds a new column to the existing `X_train_year` with the `artist_popularity` corresponding to the song's artist.** 

🚨 Make sure that the target `popularity` does not end up in `X_train_engineered` 

In [36]:
X_train_engineered = X_train_year.copy()
X_train_engineered = X_train_engineered.join(artist_popularity, on='artist')
X_train_engineered.rename(columns={'popularity': 'artist_popularity'}, inplace=True)

#### Apply the artist popularity to `X_test_year`

**📝 Similarily, create a new DataFrame `X_test_engineered` which also adds a new column to the existing `X_test_year` with the `artist_popularity` corresponding to the song's artist, computed from the training set.**

🚨**If an artist has never been seen in the training set, use the global mean popularity of all the tracks of `X_train`**

In [41]:
test_set = pd.concat([X_test, y_test], axis=1)
artist_popularity_test = test_set[['artist', 'popularity']].groupby(['artist']).mean()

X_test_engineered = X_test_year.copy()
X_test_engineered = X_test_engineered.join(artist_popularity_test, on='artist')
X_test_engineered.rename(columns={'popularity': 'artist_popularity'}, inplace=True)

# fill null values with global mean popularity in X_train
X_test_engineered['artist_popularity'] = X_test_engineered['artist_popularity'].fillna(X_train_engineered.artist_popularity.mean())

### Save your results

Run the following cell to save your results.

In [42]:
from nbresult import ChallengeResult

_ = pd.concat([X_train_engineered, X_test_engineered])

ChallengeResult("c7_feature_engineering",
    shape = _.shape,
    cols = _.columns,
    years = _.get("year"),
    popularities = _.get("artist_popularity"),
).write()

### Performance

**📝 Let's see how these features impact the performance of our model. Retrain the same baseline model on numerical values only, but adding the new features `year` and `artist_popularity`, and see how the performance is impacted. Save the performance in a `float` variable named `score_engineered`**

👉 Do not fine tune the model yet

In [43]:
X_train_engineered.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
       'instrumentalness', 'liveness', 'loudness', 'mode', 'speechiness',
       'tempo', 'valence', 'release_date', 'artist', 'year',
       'artist_popularity'],
      dtype='object')

In [44]:
X_train_baseline = X_train_engineered[['acousticness', 
                   'danceability', 
                   'duration_ms', 
                   'energy', 
                   'explicit', 
                   'instrumentalness', 
                   'liveness', 
                   'loudness',
                   'mode',
                   'speechiness',
                   'tempo',
                   'valence',
                   'year',
                   'artist_popularity',
                  ]]

In [48]:
baseline_model_eng = DummyRegressor(strategy="median")
cv_results = cross_validate(baseline_model_eng, X_train_baseline, y_train, cv=5, scoring=scoring)
score_engineered = cv_results['test_score'].mean()

In [49]:
score_engineered

-480.69396930239725

### Save your results

Run the following cell to save your results.

In [50]:
from nbresult import ChallengeResult

ChallengeResult(
    "c7_score_engineering",
    scoring=scoring,
    score_engineered=score_engineered).write()

## Pipelining

**📝 Let's create a full sklearn preprocessing pipeline called `preproc`. It should integrate our feature engineering for `year` and `artist_popularity`, as well as any other preprocessing of your choice**

**Store also the number of columns/feature after preprocessing your inputs in a variable `col_number`**

**🚨⚠️ Advice: SKIP the `ArtistPopularityTransformer` if you don't have time to do it. It is better for you to have a working pipeline rather than NO pipeline at all**

In [51]:
# 👉 Do not hesitate to reload clean new dataset if you need a fresh start
y = data.popularity
X = data.drop("popularity", axis=1)

In [52]:
# Run this cell to visualize your pipeline as you build it
from sklearn import set_config; set_config(display='diagram')

In [54]:
from sklearn.base import BaseEstimator, TransformerMixin

class ArtistPopularityTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        """
        process artist mean popularity from artists songs popularity
        process song global mean popularity
        """

        # process artist popularity
        self.artist_popularity = pd.concat([X, y],
                               axis=1,
                               join='inner').groupby(by='artist').mean()['popularity']
        # process mean popularity
        self.mean_train_popularity = y.mean()
        
        return self

    def transform(self, X, y=None):
        """
        apply artist mean popularity vs song global mean popularity to songs
        """
        
        # inject artist popularity
        X_copy = X.join(self.artist_popularity, on='artist')\
                    .rename(columns={"popularity": "artist_popularity"})\
                    .drop(columns='artist')
        
        # fills popularity of unknown artists with song global mean popularity
        X_copy['artist_popularity'].fillna(self.mean_train_popularity, inplace=True)

        return X_copy # TODO return X_copy

In [64]:
class YearTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        """
        Transform the release_dat in year (int)
        """
        X['year'] = X['release_date'].map(lambda x: x[:4]).astype(int)
        X_copy = X.drop(columns='release_date')
        return X_copy # TODO return X_copy

In [67]:
preproc = make_column_transformer(
            (MinMaxScaler(), ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo','valence']),
            (ArtistPopularityTransformer(),['artist']),
            (YearTransformer(),['release_date']), remainder='passthrough'
)

In [68]:
col_number = len(list(pd.DataFrame(preproc.fit_transform(X,y)).columns))
col_number

17

#### Save your results

Run the following cell to save your results.

In [69]:
# Print below your preproc here for the correctors
from sklearn import set_config; set_config(display='diagram')
preproc

ColumnTransformer(remainder='passthrough',
                  transformers=[('minmaxscaler', MinMaxScaler(),
                                 ['acousticness', 'danceability', 'duration_ms',
                                  'energy', 'instrumentalness', 'liveness',
                                  'loudness', 'speechiness', 'tempo',
                                  'valence']),
                                ('artistpopularitytransformer',
                                 ArtistPopularityTransformer(), ['artist']),
                                ('yeartransformer', YearTransformer(),
                                 ['release_date'])])

In [70]:
from nbresult import ChallengeResult

ChallengeResult(
    "c6_preprocessing",
    col_number=col_number
).write()

## Training

📝 Time to optimize 

- **Add an estimator to your pipeline (only from scikit-learn)** 

- **Train your pipeline and fine-tune (optimize) your estimator to get the best prediction score**

- **You must create 2 pipelines (one with a linear model, one with an ensemble model)**

Then, 

- Save your two best 5-time cross-validated scores as _float_: `score_linear` and `score_ensemble`

- Save your two best trained pipelines as _Pipeline_ objects: `pipe_linear` and `pipe_ensemble`

### Linear

### Ensemble

### Save your results

Run the following cell to save your results.

In [ ]:
# Print below your best pipe for correction purpose
from sklearn import set_config; set_config(display='diagram')
pipe_linear

In [ ]:
# Print below your best pipe for correction purpose
pipe_ensemble

In [ ]:
from nbresult import ChallengeResult

ChallengeResult("c8_c9_c11_c13_model_tuning",
    scoring = scoring,
    score_linear=score_linear,
    score_ensemble=score_ensemble).write()

## API 

Time to put a pipeline in production!

👉 Go to https://github.com/lewagon/data-certification-api and follow instructions

**This final part is independent from the above notebook**